In [47]:
from DSSATTools import (
    crop, # module that contains the classes for the different crops.
    WeatherStation, # class needed to define the Weather. 
    SoilProfile, SoilLayer, # classes that define the entire soil profile, and the soil layers within the soil profile
    filex, # module that contains the clases for the FileX sections.
    DSSAT # the simulation environment class.
)
import pandas as pd
from datetime import datetime
import tempfile
import os

In [1]:
# --- 1. SET YOUR INPUTS HERE ---
#---------------------------------
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"
OUTPUT_FILENAME = "MY_GENERATED_SOIL.SOL"
START_DATE = "2013-01-01"
END_DATE = "2013-12-31"
API_KEY = "eddbc7c0c9e63e225934add809624c6e" # This not orginal key
#---------------------------------

In [ ]:

# creating a Weather file  from a realtime source
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from collections import defaultdict
from typing import Optional, Dict

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Fetches O3 data from OpenWeatherMap API."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("Warning: OpenWeatherMap API key is missing.")
        return pd.DataFrame()
    
    print("Fetching OpenWeatherMap O3 data...")
    try:
        start_ts = int(pd.to_datetime(start_str).timestamp())
        end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()
        data = response.json().get('list', [])

        if not data:
            print("...O3 data fetched, but no records returned for the period.")
            return pd.DataFrame()

        daily_data = defaultdict(list)
        for entry in data:
            local_date = datetime.fromtimestamp(entry['dt']).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
        df = pd.DataFrame(records)
        df['date'] = pd.to_datetime(df['date'])
        print("...O3 data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching O3 data: {e}. Proceeding without it.")
        return pd.DataFrame()

def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    """Fetches NASA POWER weather data."""
    print("Fetching NASA POWER data...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        df = df.replace([-999, -99], np.nan)
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
            "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
            "RH2M": "rhum", "EVPTRNS": "evap"
        })
        print("...NASA data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"FATAL: Error fetching NASA POWER data: {e}")
        return pd.DataFrame()

def create_weather_file(
    api_key: str,
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    site_name: str = "SITE",
    insi: str = "XXXX",
    elev: Optional[float] = None
) -> Optional[pd.DataFrame]:
    """
    Fetches weather data and writes a standard DSSAT-compatible .WTH file.
    """
    weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
    if weather_df.empty:
        print("Could not fetch base weather data from NASA POWER. Aborting.")
        return None

    o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
    df = weather_df
    if not o3_df.empty:
        df = pd.merge(df, o3_df, on='date', how='left')

    if 'ozon7' in df.columns:
        df['ozon7'] = df['ozon7'].fillna(50.0)
    else:
        df['ozon7'] = 50.0
    
    df['dco2'] = 421.0

    df["wind"] = df["wind"] * 86.4
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
    # --- CHANGE #1: Revert to the standard YRDOY (YYYYJJJ) format ---
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    df_out = df.fillna(-99)
    
    # --- CHANGE #2: Update the list of columns to use YRDOY ---
    output_cols = ["YRDOY","srad","tmax","tmin","rain","rhum","wind","dewp","par","vapr","dco2","ozon7","evap"]
    
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    
    year_2_digit = pd.to_datetime(start_date).strftime('%y')
    wth_file = os.path.join(output_dir, f"{insi}{year_2_digit}01.WTH")
    print(f"\nWriting weather data to: {wth_file}")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},NASA\n\n")
        f.write("@ INSI      LAT     LONG    ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else -99:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
        f.write("@DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        
        # --- CHANGE #3: Write the date as a 7-digit integer ---
        # for _, row in df_out[output_cols].iterrows():
        #     f.write(f"{int(row.YRDOY):5d}"
        #             f"{row.srad:6.1f}{row.tmax:6.1f}{row.tmin:6.1f}{row.rain:6.1f}"
        #             f"{row.rhum:6.1f}{row.wind:6.1f}{row.dewp:6.1f}{row.par:6.1f}"
        #             f"{row.vapr:6.3f}{row.dco2:6.1f}{row.ozon7:6.0f}{row.evap:6.1f}\n")
        for _, row in df_out[output_cols].iterrows():
            f.write(f"{row.YRDOY:>5s}"  # ensure "24001", not 24001
                    f"{row.srad:6.1f}{row.tmax:6.1f}{row.tmin:6.1f}{row.rain:6.1f}"
                    f"{row.rhum:6.1f}{row.wind:6.1f}{row.dewp:6.1f}{row.par:6.1f}"
                    f"{row.vapr:6.3f}{row.dco2:6.1f}{row.ozon7:6.0f}{row.evap:6.1f}\n")
    
    print("✅ File created successfully.")
    return df_out

if __name__ == "__main__":
    final_data = create_weather_file(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        site_name="CHENNAI",
        insi="CHEN",
        elev=6.0
    )

Fetching NASA POWER data...
...NASA data fetched successfully.
Fetching OpenWeatherMap O3 data...
...O3 data fetched, but no records returned for the period.

Writing weather data to: weather\CHEN1301.WTH
✅ File created successfully.


In [ ]:
# # creating a Weather file  from a realtime source
# import requests
# import pandas as pd
# import numpy as np
# import os
# from datetime import datetime
# from collections import defaultdict
# from typing import Optional, Dict

# def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
#     """Fetches O3 data from OpenWeatherMap API."""
#     if not api_key or api_key == "YOUR_API_KEY_HERE":
#         print("Warning: OpenWeatherMap API key is missing.")
#         return pd.DataFrame()
    
#     print("Fetching OpenWeatherMap O3 data...")
#     try:
#         start_ts = int(pd.to_datetime(start_str).timestamp())
#         end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
#         api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
#         response = requests.get(api_url, timeout=30)
#         response.raise_for_status()
#         data = response.json().get('list', [])

#         if not data:
#             print("...O3 data fetched, but no records returned for the period.")
#             return pd.DataFrame()

#         daily_data = defaultdict(list)
#         for entry in data:
#             local_date = datetime.fromtimestamp(entry['dt']).date()
#             daily_data[local_date].append(entry['components'].get('o3', 0))

#         records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
#         df = pd.DataFrame(records)
#         df['date'] = pd.to_datetime(df['date'])
#         print("...O3 data fetched successfully.")
#         return df
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching O3 data: {e}. Proceeding without it.")
#         return pd.DataFrame()

# def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
#     """Fetches NASA POWER weather data."""
#     print("Fetching NASA POWER data...")
#     try:
#         url = "https://power.larc.nasa.gov/api/temporal/daily/point"
#         params = {
#             "start": pd.to_datetime(start).strftime("%Y%m%d"),
#             "end": pd.to_datetime(end).strftime("%Y%m%d"),
#             "latitude": lat, "longitude": lon, "community": "AG",
#             "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
#             "format": "JSON"
#         }
#         response = requests.get(url, params=params, timeout=30)
#         response.raise_for_status()
        
#         data = response.json()["properties"]["parameter"]
#         df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
#         df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
#         df = df.replace([-999, -99], np.nan)
#         df = df.rename(columns={
#             "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
#             "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
#             "RH2M": "rhum", "EVPTRNS": "evap"
#         })
#         print("...NASA data fetched successfully.")
#         return df
#     except requests.exceptions.RequestException as e:
#         print(f"FATAL: Error fetching NASA POWER data: {e}")
#         return pd.DataFrame()

# def create_weather_file(
#     api_key: str,
#     lat: float,
#     lon: float,
#     start_date: str,
#     end_date: str,
#     site_name: str = "SITE",
#     insi: str = "XXXX",
#     elev: Optional[float] = None
# ) -> Optional[pd.DataFrame]:
#     """
#     Fetches weather data and writes a standard DSSAT-compatible .WTH file.
#     """
#     weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
#     if weather_df.empty:
#         print("Could not fetch base weather data from NASA POWER. Aborting.")
#         return None

#     o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
#     df = weather_df
#     if not o3_df.empty:
#         df = pd.merge(df, o3_df, on='date', how='left')

#     if 'ozon7' in df.columns:
#         df['ozon7'] = df['ozon7'].fillna(50.0)
#     else:
#         df['ozon7'] = 50.0
    
#     df['dco2'] = 421.0

#     df["wind"] = df["wind"] * 86.4
#     df["month"] = df["date"].dt.month
#     monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
#     df["par"] = df["month"].map(monthly_srad_avg)
#     df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
#     # --- CHANGE #1: Revert to the standard YRDOY (YYJJJ) format ---
#     df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
#     tav = ((df["tmax"] + df["tmin"]) / 2).mean()
#     amp = (df["tmax"].mean() - df["tmin"].mean())

#     df_out = df.fillna(-99)
    
#     # --- CHANGE #2: Update the list of columns to use YRDOY ---
#     output_cols = ["YRDOY","srad","tmax","tmin","rain","rhum","wind","dewp","par","vapr","dco2","ozon7","evap"]
    
#     output_dir = "weather"
#     os.makedirs(output_dir, exist_ok=True)
    
#     year_2_digit = pd.to_datetime(start_date).strftime('%y')
#     wth_file = os.path.join(output_dir, f"{insi}{year_2_digit}01.WTH")
#     print(f"\nWriting weather data to: {wth_file}")

#     with open(wth_file, "w") as f:
#         f.write(f"*WEATHER DATA : {site_name},NASA\n\n")
#         f.write("@ INSI      LAT     LONG    ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
#         f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else -99:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
#         f.write("@DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        
#         # --- CHANGE #3: Write the date as a 7-digit integer ---
#         for _, row in df_out[output_cols].iterrows():
#             f.write(f"{int(row.YRDOY):7d} {row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
#                     f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} {row.dewp:5.1f} "
#                     f"{row.par:6.1f} {row.vapr:5.2f} {row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")
    
#     print("✅ File created successfully.")
#     return df_out

# if __name__ == "__main__":
#     final_data = create_weather_file(
#         api_key=API_KEY,
#         lat=LATITUDE,
#         lon=LONGITUDE,
#         start_date=START_DATE,
#         end_date=END_DATE,
#         site_name="CHENNAI",
#         insi="CHEN",
#         elev=6.0
#     )

In [ ]:

# creating a Weather file  from a realtime source
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from collections import defaultdict
from typing import Optional, Dict

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Fetches O3 data from OpenWeatherMap API."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("Warning: OpenWeatherMap API key is missing.")
        return pd.DataFrame()
    
    print("Fetching OpenWeatherMap O3 data...")
    try:
        start_ts = int(pd.to_datetime(start_str).timestamp())
        end_ts = int(pd.to_datetime(end_str).replace(hour=23, minute=59).timestamp())
        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()
        data = response.json().get('list', [])

        if not data:
            print("...O3 data fetched, but no records returned for the period.")
            return pd.DataFrame()

        daily_data = defaultdict(list)
        for entry in data:
            local_date = datetime.fromtimestamp(entry['dt']).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'ozon7': np.mean(vals)} for date, vals in sorted(daily_data.items())]
        df = pd.DataFrame(records)
        df['date'] = pd.to_datetime(df['date'])
        print("...O3 data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching O3 data: {e}. Proceeding without it.")
        return pd.DataFrame()

def _fetch_nasa_weather(lat: float, lon: float, start: str, end: str) -> pd.DataFrame:
    """Fetches NASA POWER weather data."""
    print("Fetching NASA POWER data...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()["properties"]["parameter"]
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        df = df.replace([-999, -99], np.nan)
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
            "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
            "RH2M": "rhum", "EVPTRNS": "evap"
        })
        print("...NASA data fetched successfully.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"FATAL: Error fetching NASA POWER data: {e}")
        return pd.DataFrame()

def create_weather_file(
    api_key: str,
    lat: float,
    lon: float,
    start_date: str,
    end_date: str,
    site_name: str = "SITE",
    insi: str = "XXXX",
    elev: Optional[float] = None
) -> Optional[pd.DataFrame]:
    """
    Fetches weather data and writes a standard DSSAT-compatible .WTH file.
    """
    weather_df = _fetch_nasa_weather(lat, lon, start_date, end_date)
    if weather_df.empty:
        print("Could not fetch base weather data from NASA POWER. Aborting.")
        return None

    o3_df = get_o3(api_key, lat, lon, start_date, end_date)
    
    df = weather_df
    if not o3_df.empty:
        df = pd.merge(df, o3_df, on='date', how='left')

    if 'ozon7' in df.columns:
        df['ozon7'] = df['ozon7'].fillna(50.0)
    else:
        df['ozon7'] = 50.0
    
    df['dco2'] = 421.0

    df["wind"] = df["wind"] * 86.4
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    
    # --- CHANGE #1: Revert to the standard YRDOY (YYYYJJJ) format ---
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    df_out = df.fillna(-99)
    
    # --- CHANGE #2: Update the list of columns to use YRDOY ---
    output_cols = ["YRDOY","srad","tmax","tmin","rain","rhum","wind","dewp","par","vapr","dco2","ozon7","evap"]
    
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    
    year_2_digit = pd.to_datetime(start_date).strftime('%y')
    wth_file = os.path.join(output_dir, f"{insi}{year_2_digit}01.WTH")
    print(f"\nWriting weather data to: {wth_file}")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},NASA\n\n")
        f.write("@ INSI      LAT     LONG    ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else -99:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
        f.write("@DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        
        # --- CHANGE #3: Write the date as a 7-digit integer ---
        for _, row in df_out[output_cols].iterrows():
            f.write(f"{int(row.YRDOY):7d} {row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
                    f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} {row.dewp:5.1f} "
                    f"{row.par:6.1f} {row.vapr:5.2f} {row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")
    
    print("✅ File created successfully.")
    return df_out

if __name__ == "__main__":
    final_data = create_weather_file(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        site_name="CHENNAI",
        insi="CHEN",
        elev=6.0
    )

Fetching NASA POWER data...
...NASA data fetched successfully.
Fetching OpenWeatherMap O3 data...
...O3 data fetched successfully.

Writing weather data to: weather\CHEN2401.WTH
✅ File created successfully.


In [ ]:
# # Creating Soil profile from realtime 
# import logging
# import sys
# import time
# from typing import List

# import numpy as np
# import pandas as pd
# import requests



# # Import the necessary classes AND the helper function from DSSATTools
# from DSSATTools import SoilProfile, SoilLayer
# from DSSATTools.soil import estimate_from_texture

# # --- 2. HELPER FUNCTIONS ---

# # --- Configuration Constants ---
# SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
# SOILGRIDS_PROPERTIES = [
#     "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
#     "soc", "phh2o", "cec", "nitrogen"
# ]

# def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
#     """Fetches soil property data robustly with a retry mechanism."""
#     logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
#     base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
#     all_properties_data = []
#     max_retries = 4
#     initial_retry_delay_seconds = 5

#     for prop in properties:
#         logging.info(f"--> Fetching property: {prop}")
#         params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
#         for attempt in range(max_retries):
#             try:
#                 response = requests.get(base_url, params=params, timeout=45)
#                 response.raise_for_status()
#                 data = response.json()
#                 layers = data.get('properties', {}).get('layers', [])
#                 if not layers:
#                     logging.warning(f"No layer data for {prop}. Skipping.")
#                     break
#                 parsed_data = {
#                     depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
#                     for depth_data in layers[0]['depths']
#                 }
#                 prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
#                 all_properties_data.append(prop_df)
#                 time.sleep(1)
#                 break
#             except requests.exceptions.HTTPError as e:
#                 if e.response.status_code == 429 and attempt < max_retries - 1:
#                     wait_time = initial_retry_delay_seconds * (2 ** attempt)
#                     logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
#                     time.sleep(wait_time)
#                 else:
#                     logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
#                     break
#             except (requests.exceptions.RequestException, ValueError, KeyError) as e:
#                 logging.error(f"Failed on property '{prop}': {e}. Skipping.")
#                 break
#     if not all_properties_data:
#         raise ValueError("Failed to fetch any data from SoilGrids.")
#     final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
#     for prop in properties:
#         if prop not in final_df.columns:
#             final_df[prop] = np.nan
#     return final_df

# # --- UPDATED processing function ---
# def process_and_estimate_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Converts raw SoilGrids data and uses the DSSATTools `estimate_from_texture` 
#     function to calculate hydraulic properties for each layer.
#     """
#     logging.info("Processing raw data and estimating hydraulic properties with DSSATTools...")
    
#     # Initialize DataFrame with basic info
#     dssat_df = pd.DataFrame(index=raw_df.index)
#     dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
#     dssat_df['SLOC'] = raw_df['soc'] / 100
#     dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
#     dssat_df['SLHW'] = raw_df['phh2o'] / 10
#     dssat_df['SCEC'] = raw_df['cec'] / 10
#     dssat_df['SLCF'] = raw_df['cfvo'] / 10
    
#     # Define root growth factor
#     rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
#     dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    
#     # Loop through each layer to estimate properties
#     for index, row in raw_df.iterrows():
#         slcl = row['clay'] / 10
#         slsi = row['silt'] / 10
#         sbdm = row['bdod'] / 100
        
#         dssat_df.loc[index, 'SLCL'] = slcl
#         dssat_df.loc[index, 'SLSI'] = slsi
#         dssat_df.loc[index, 'SBDM'] = sbdm

#         if pd.notna(slcl) and pd.notna(slsi) and pd.notna(sbdm):
#             # Use the library's built-in function to get consistent properties
#             estimated_props = estimate_from_texture(slcl=slcl, slsi=slsi, sbdm=sbdm)
#             dssat_df.loc[index, 'SLLL'] = estimated_props['slll']
#             dssat_df.loc[index, 'SDUL'] = estimated_props['sdul']
#             dssat_df.loc[index, 'SSAT'] = estimated_props['ssat']
#             dssat_df.loc[index, 'SSKS'] = estimated_props['ssks']
#         else:
#             logging.warning(f"Missing texture/bulk density for layer {index}. Cannot estimate properties.")

#     return dssat_df

# def get_texture_class(sand: float, silt: float, clay: float) -> str:
#     """Determines USDA soil texture class from sand, silt, clay percentages."""
#     if (silt + 1.5 * clay) < 15: return 'S'
#     if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
#     if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
#     if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
#     if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
#     if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
#     if (silt >= 50 and clay < 12): return 'SI'
#     if (silt >= 80 and clay < 12): return 'SI'
#     if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
#     if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
#     if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
#     if (clay >= 35) and (sand > 45): return 'SC'
#     if (clay >= 40) and (silt >= 40): return 'SIC'
#     if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
#     return 'UNKNOWN'

# def write_sol_file_with_dssattools(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
#     """
#     Builds a SoilProfile object from the DataFrame and writes it to a file
#     using the DSSATTools library classes.
#     """
#     logging.info(f"Building SoilProfile object with DSSATTools...")
    
#     soil_layers = []
#     for _, row in df.iterrows():
#         kwargs = {k.lower(): v for k, v in row.to_dict().items()}
#         kwargs = {k: v for k, v in kwargs.items() if pd.notna(v)}
#         soil_layers.append(SoilLayer(**kwargs))

#     top_layer = df.iloc[0]
#     sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
#     texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
#     soil_id = f"SG{str(int(abs(lat*100))):0>4}{str(int(abs(lon*100))):0>4}"
    
#     soil_profile = SoilProfile(
#         name=soil_id,
#         soil_series_name='SoilGrids Auto-Generated Profile',
#         soil_clasification=texture,
#         site='SoilGrids',
#         country=country,
#         lat=lat,
#         long=lon,
#         salb=0.13, slu1=6.0, slro=75.0, sldr=-99, slnf=1.0, slpf=1.0,
#         table=soil_layers
#     )
    
#     logging.info(f"Writing DSSAT .SOL file to {filename}")
#     with open(filename, 'w') as f:
#         f.write(soil_profile._write_sol())


# # --- 3. MAIN EXECUTION LOGIC ---
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

# try:
#     raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
    
#     # Use the new, unified processing and estimation function
#     final_dssat_profile = process_and_estimate_dssat_format(raw_soil_data)

#     print("\n--- Final Processed DSSAT Data ---")
#     print(final_dssat_profile.round(3).to_string())

#     write_sol_file_with_dssattools(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
#     print(f"\n✅ Successfully created soil file using DSSATTools: {OUTPUT_FILENAME}")

#     print(f"\nVerifying '{OUTPUT_FILENAME}'...")
#     soil_id_to_check = f"SG{str(int(abs(LATITUDE*100))):0>4}{str(int(abs(LONGITUDE*100))):0>4}"
#     verified_soil = SoilProfile.from_file(soil_id_to_check, OUTPUT_FILENAME)
#     print("✅ Verification successful! The file can be read by DSSATTools.")

# except Exception as e:
#     logging.error(f"Script failed: {e}")

In [48]:
# Creating Soil profile from realtime 
import logging
import sys
import time
from typing import List

import numpy as np
import pandas as pd
import requests

# Import the necessary classes AND the helper function from DSSATTools
from DSSATTools import SoilProfile, SoilLayer
from DSSATTools.soil import estimate_from_texture

# --- 2. HELPER FUNCTIONS ---

# --- Configuration Constants ---
SOILGRIDS_DEPTHS = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]
SOILGRIDS_PROPERTIES = [
    "sand", "silt", "clay", "bdod", "cfvo", "wv0033", "wv1500",
    "soc", "phh2o", "cec", "nitrogen"
]

def fetch_soilgrids_data_robust(latitude: float, longitude: float, properties: List[str], depths: List[str]) -> pd.DataFrame:
    """Fetches soil property data robustly with a retry mechanism."""
    logging.info(f"Fetching SoilGrids data for lat={latitude}, lon={longitude}")
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    all_properties_data = []
    max_retries = 4
    initial_retry_delay_seconds = 5

    for prop in properties:
        logging.info(f"--> Fetching property: {prop}")
        params = {'lon': longitude, 'lat': latitude, 'property': prop, 'depth': depths, 'value': 'mean'}
        for attempt in range(max_retries):
            try:
                response = requests.get(base_url, params=params, timeout=45)
                response.raise_for_status()
                data = response.json()
                layers = data.get('properties', {}).get('layers', [])
                if not layers:
                    logging.warning(f"No layer data for {prop}. Skipping.")
                    break
                parsed_data = {
                    depth_data['label']: depth_data.get('values', {}).get('mean', np.nan)
                    for depth_data in layers[0]['depths']
                }
                prop_df = pd.DataFrame.from_dict(parsed_data, orient='index', columns=[prop])
                all_properties_data.append(prop_df)
                time.sleep(1)
                break
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429 and attempt < max_retries - 1:
                    wait_time = initial_retry_delay_seconds * (2 ** attempt)
                    logging.warning(f"Rate limit hit for '{prop}'. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    logging.error(f"HTTP error for '{prop}': {e}. It will be missing.")
                    break
            except (requests.exceptions.RequestException, ValueError, KeyError) as e:
                logging.error(f"Failed on property '{prop}': {e}. Skipping.")
                break
    if not all_properties_data:
        raise ValueError("Failed to fetch any data from SoilGrids.")
    final_df = pd.concat(all_properties_data, axis=1).reindex(depths)
    for prop in properties:
        if prop not in final_df.columns:
            final_df[prop] = np.nan
    return final_df

def process_and_estimate_dssat_format(raw_df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts raw SoilGrids data and uses the DSSATTools `estimate_from_texture` 
    function to calculate hydraulic properties for each layer.
    """
    logging.info("Processing raw data and estimating hydraulic properties with DSSATTools...")
    
    dssat_df = pd.DataFrame(index=raw_df.index)
    dssat_df['SLB'] = [int(label.split('-')[1].replace('cm', '')) for label in raw_df.index]
    dssat_df['SLOC'] = raw_df['soc'] / 100
    dssat_df['SLNI'] = raw_df['nitrogen'] / 1000
    dssat_df['SLHW'] = raw_df['phh2o'] / 10
    dssat_df['SCEC'] = raw_df['cec'] / 10
    dssat_df['SLCF'] = raw_df['cfvo'] / 10
    
    rgf_map = {'0-5cm': 1.0, '5-15cm': 1.0, '15-30cm': 1.0, '30-60cm': 0.8, '60-100cm': 0.5, '100-200cm': 0.2}
    dssat_df['SRGF'] = dssat_df.index.map(rgf_map)
    
    for index, row in raw_df.iterrows():
        slcl = row['clay'] / 10
        slsi = row['silt'] / 10
        sbdm = row['bdod'] / 100
        
        dssat_df.loc[index, 'SLCL'] = slcl
        dssat_df.loc[index, 'SLSI'] = slsi
        dssat_df.loc[index, 'SBDM'] = sbdm

        if pd.notna(slcl) and pd.notna(slsi) and pd.notna(sbdm):
            estimated_props = estimate_from_texture(slcl=slcl, slsi=slsi, sbdm=sbdm)
            dssat_df.loc[index, 'SLLL'] = estimated_props['slll']
            dssat_df.loc[index, 'SDUL'] = estimated_props['sdul']
            dssat_df.loc[index, 'SSAT'] = estimated_props['ssat']
            dssat_df.loc[index, 'SSKS'] = estimated_props['ssks']
        else:
            logging.warning(f"Missing texture/bulk density for layer {index}. Cannot estimate properties.")

    return dssat_df

def get_texture_class(sand: float, silt: float, clay: float) -> str:
    """Determines USDA soil texture class from sand, silt, clay percentages."""
    if (silt + 1.5 * clay) < 15: return 'S'
    if (silt + 1.5 * clay) >= 15 and (silt + 2 * clay) < 30: return 'LS'
    if (clay >= 7 and clay < 20) and (sand > 52) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay < 7) and (silt < 50) and ((silt + 2 * clay) >= 30): return 'SL'
    if (clay >= 7 and clay < 27) and (silt >= 28 and silt < 50) and (sand <= 52): return 'L'
    if (silt >= 50 and clay >= 12 and clay < 27): return 'SIL'
    if (silt >= 50 and clay < 12): return 'SI'
    if (silt >= 80 and clay < 12): return 'SI'
    if (clay >= 20 and clay < 35) and (silt < 28) and (sand > 45): return 'SCL'
    if (clay >= 27 and clay < 40) and (sand > 20 and sand <= 45): return 'CL'
    if (clay >= 27 and clay < 40) and (sand <= 20): return 'SICL'
    if (clay >= 35) and (sand > 45): return 'SC'
    if (clay >= 40) and (silt >= 40): return 'SIC'
    if (clay >= 40) and (sand <= 45) and (silt < 40): return 'C'
    return 'UNKNOWN'

def write_sol_file_with_dssattools(df: pd.DataFrame, lat: float, lon: float, country: str, filename: str):
    """
    Builds a SoilProfile object from the DataFrame and writes it to a file
    using the DSSATTools library classes.
    """
    logging.info(f"Building SoilProfile object with DSSATTools...")
    
    soil_layers = []
    for _, row in df.iterrows():
        kwargs = {k.lower(): v for k, v in row.to_dict().items()}
        kwargs = {k: v for k, v in kwargs.items() if pd.notna(v)}
        soil_layers.append(SoilLayer(**kwargs))

    top_layer = df.iloc[0]
    sand_pct = 100 - top_layer.get('SLCL', 0) - top_layer.get('SLSI', 0)
    texture = get_texture_class(sand_pct, top_layer.get('SLSI', 0), top_layer.get('SLCL', 0))
    soil_id = f"SG{str(int(abs(lat*100))):0>4}{str(int(abs(lon*100))):0>4}"
    
    # --- START OF SLDR FIX ---
    # 1. Create a lookup table for SLDR based on soil texture class.
    # These values are based on typical ranges found in the SOIL.SOL reference file.
    SLDR_LOOKUP = {
        'S': 0.75, 'LS': 0.70, 'SL': 0.65,  # Sandy soils (high drainage)
        'L': 0.60, 'SIL': 0.55, 'SI': 0.55,   # Loam soils (medium drainage)
        'SCL': 0.50, 'CL': 0.50, 'SICL': 0.45, # Clay Loam soils
        'SC': 0.40, 'SIC': 0.35, 'C': 0.30,   # Clay soils (low drainage)
    }

    # 2. Look up the sldr_value, with a safe default of 0.5
    sldr_value = SLDR_LOOKUP.get(texture, 0.5)
    logging.info(f"Determined soil texture as '{texture}', setting SLDR to {sldr_value}")
    # --- END OF SLDR FIX ---
    
    soil_profile = SoilProfile(
        name=soil_id,
        soil_series_name='SoilGrids Auto-Generated Profile',
        soil_clasification=texture,
        site='SoilGrids',
        country=country,
        lat=lat,
        long=lon,
        salb=0.13, slu1=6.0, slro=75.0, 
        sldr=sldr_value, # <-- 3. Use the dynamically determined sldr_value instead of -99
        slnf=1.0, slpf=1.0,
        table=soil_layers
    )
    
    logging.info(f"Writing DSSAT .SOL file to {filename}")
    with open(filename, 'w') as f:
        f.write(soil_profile._write_sol())

# --- 3. MAIN EXECUTION LOGIC ---
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', stream=sys.stdout)

    try:
        raw_soil_data = fetch_soilgrids_data_robust(LATITUDE, LONGITUDE, SOILGRIDS_PROPERTIES, SOILGRIDS_DEPTHS)
        
        final_dssat_profile = process_and_estimate_dssat_format(raw_soil_data)

        print("\n--- Final Processed DSSAT Data ---")
        print(final_dssat_profile.round(3).to_string())

        write_sol_file_with_dssattools(final_dssat_profile, LATITUDE, LONGITUDE, COUNTRY, OUTPUT_FILENAME)
        print(f"\n✅ Successfully created soil file using DSSATTools: {OUTPUT_FILENAME}")

        print(f"\nVerifying '{OUTPUT_FILENAME}'...")
        soil_id_to_check = f"SG{str(int(abs(LATITUDE*100))):0>4}{str(int(abs(LONGITUDE*100))):0>4}"
        verified_soil = SoilProfile.from_file(soil_id_to_check, OUTPUT_FILENAME)
        print("✅ Verification successful! The file can be read by DSSATTools.")

    except Exception as e:
        logging.error(f"Script failed: {e}")


2025-10-02 16:27:57,764 - INFO - Fetching SoilGrids data for lat=12.3811, lon=78.9366
2025-10-02 16:27:57,765 - INFO - --> Fetching property: sand
2025-10-02 16:28:00,376 - INFO - --> Fetching property: silt
2025-10-02 16:28:02,772 - INFO - --> Fetching property: clay
2025-10-02 16:28:04,903 - INFO - --> Fetching property: bdod
2025-10-02 16:28:14,458 - INFO - --> Fetching property: cfvo
2025-10-02 16:28:16,649 - INFO - --> Fetching property: wv0033
2025-10-02 16:28:28,238 - INFO - --> Fetching property: wv1500
2025-10-02 16:28:31,098 - INFO - --> Fetching property: soc
2025-10-02 16:28:34,591 - INFO - --> Fetching property: phh2o
2025-10-02 16:28:40,598 - INFO - --> Fetching property: cec
2025-10-02 16:28:44,401 - INFO - --> Fetching property: nitrogen
2025-10-02 16:28:47,190 - INFO - Processing raw data and estimating hydraulic properties with DSSATTools...

--- Final Processed DSSAT Data ---
           SLB  SLOC   SLNI  SLHW  SCEC  SLCF  SRGF  SLCL  SLSI  SBDM   SLLL   SDUL   SSAT  

In [ ]:
# getting realtime forecast data
import requests
import datetime
from typing import Union

def get_rain_forecast(api_key: str, latitude: float, longitude: float, desc: bool = True) -> Union[str, float]:
    """
    Fetches the next day's rain forecast for a specific lat/lon.

    Args:
        api_key: Your OpenWeatherMap API key.
        latitude: The latitude of the location.
        longitude: The longitude of the location.
        desc: If True, returns a formatted string description.
              If False, returns the total rain in mm as a float.

    Returns:
        A string or a float depending on the 'desc' parameter.
    """
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        'lat': latitude,
        'lon': longitude,
        'appid': api_key,
        'units': 'metric'
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        forecast_data = response.json()
        
        location_name = forecast_data.get('city', {}).get('name', f"Lat/Lon: {latitude},{longitude}")
        tomorrow = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        
        descriptions = []
        total_rain_mm = 0.0

        for forecast in forecast_data['list']:
            if tomorrow in forecast['dt_txt']:
                rain_mm = forecast.get('rain', {}).get('3h', 0)
                total_rain_mm += rain_mm

                if rain_mm > 0:
                    forecast_time = forecast['dt_txt'].split(' ')[1]
                    weather_description = forecast['weather'][0]['description']
                    pop = forecast.get('pop', 0) * 100
                    desc_line = (f" -> At {forecast_time}: {rain_mm} mm of rain expected. "
                                 f"(Chance: {pop:.0f}%, Forecast: {weather_description})")
                    descriptions.append(desc_line)

        # Return value based on the 'desc' parameter
        if not desc:
            return round(total_rain_mm, 2)
        else:
            if not descriptions:
                return f"No measurable rain is expected for {location_name} tomorrow. ☀️"
            else:
                header = f"Rain forecast for {location_name} on {tomorrow}:\n"
                return header + "\n".join(descriptions)

    except requests.exceptions.RequestException as e:
        return f"Error fetching data: {e}" if desc else -1.0
    except KeyError:
        return "Error parsing data." if desc else -1.0

# --- Main execution ---
if __name__ == "__main__":

        # --- Test Case 1: With Description (desc=True) ---
    print("--- Fetching with Description (the default) ---")
    description_output = get_rain_forecast(API_KEY, LATITUDE, LONGITUDE, desc=True)
    print(description_output)
    
    print("\n" + "="*50 + "\n")

    # --- Test Case 2: Numerical Value Only (desc=False) ---
    print("--- Fetching Total Rain Value Only ---")
    value_output = get_rain_forecast(API_KEY, LATITUDE, LONGITUDE, desc=False)
    print(f"Total expected rain for tomorrow: {value_output} mm")

--- Fetching with Description (the default) ---
Rain forecast for Chengam on 2025-10-02:
 -> At 12:00:00: 0.19 mm of rain expected. (Chance: 20%, Forecast: light rain)
 -> At 15:00:00: 0.4 mm of rain expected. (Chance: 43%, Forecast: light rain)
 -> At 18:00:00: 2.35 mm of rain expected. (Chance: 89%, Forecast: light rain)
 -> At 21:00:00: 0.8 mm of rain expected. (Chance: 58%, Forecast: light rain)


--- Fetching Total Rain Value Only ---
Total expected rain for tomorrow: 3.74 mm
